In [ ]:
using Catalyst
using OrdinaryDiffEq
using JumpProcesses
using DiffEqCallbacks
using Random
using Distributions
using LogExpFunctions
using PyPlot

In [ ]:
function signal_conc(t)
    return 4e-3 * logistic(10t - 30) * logistic(60 - 10t)
end

In [ ]:
tt = 0:0.01:20
plt.plot(tt, signal_conc.(tt))

In [ ]:
rn = @reaction_network begin
    # Use a Michaelis-Menten reaction to activate the gene
    mm(signal_conc(t), σ_on, K_m),  G_off --> G_on 
    σ_off , G_on --> G_off       # Gene deactivation rate
    
    ρ_m, G_on --> G_on + M       # mRNA transcription 
    δ_m, M --> 0                 # mRNA degradation
    ρ_p, M --> M + P         # volume-dependent translation
    δ_p, P --> 0                 # protein degradation 
end

In [ ]:
p = [ # Telegraph switching rates
      :σ_on => 1,
      :K_m => 5e-3,
      :σ_off => 0.2,        

      # mRNA production & degradation rates
      :ρ_m => 3,             
      :δ_m => 0.5,
    
      # Protein production & degradation rates
      :ρ_p => 20,             
      :δ_p => 1
]

u0 = [  
       :G_on => 0,
       :G_off => 1,
       :M => 0,
       :P => 0 
]

tmax = 20.
doubling_time = 7

In [ ]:
function divide!(int)   
    # mRNA and proteins are divided randomly between the daughters
    # We can access variables using bracket notation
    int[:M] = rand(Binomial(int[:M], 0.5))
    int[:P] = rand(Binomial(int[:P], 0.5))
end

cb_divide = PeriodicCallback(divide!, doubling_time);

In [ ]:
jinp = JumpInputs(rn, u0, (0, tmax), p)

In [ ]:
prob = JumpProblem(jinp, Direct())

In [ ]:
sol = solve(prob, Tsit5(); callback=cb_divide);

In [ ]:
plt.plot(sol.t, sol[:G_on])

In [ ]:
plt.plot(sol.t, sol[:M])

In [ ]:
plt.plot(sol.t, sol[:P])